In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-dh6b2qm6/unsloth_aafeadb33c5d4d31ad43d394591d32d4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-dh6b2qm6/unsloth_aafeadb33c5d4d31ad43d394591d32d4
  Resolved https://github.com/unslothai/unsloth.git to commit 3085f4c3daacc63939e78e3c87759d0d03c5a71f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install xformers

In [3]:
!pip install transformers

In [4]:
!pip install trl

In [5]:
from datasets import load_dataset
import torch

In [6]:
data = load_dataset('imdb', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
print(data.shape)
print(data[129])

(25000, 2)
{'text': 'Ah, Bait. How do I hate thee? Let me count the ways. 1. You try to be funny, but are corny and unenjoyable; every joke is predictable and expected, and when it comes, does not inspire laughter. Instead, I want to hurl. 2. You try to be dramatic, but are unbelievable; the woman overacts to a terrible degree, and the "bad guy" looks like Bill Gates, and is about as scary as...well, Bill Gates. (Just try to imagine Bill Gates trying to intimidate somebody with a gun. Doesn\'t work, does it? A lawyer, maybe, but not a gun. Doesn\'t fit.) As for Jamie Foxx, well, just watching him try to deliver a dramatic and heartfelt dialogue is ludicrous, and makes me want to hurl. 3. You try to be action-packed, but instead are dull and dragging too many times. And when the action heats up, the tripod for the camera must have been lost, for the scenes wobble more than those in The Blair Witch Project, and I find myself nauseated, and once again I want to hurl. 4. You try to be a go

##Using Unsloth

In [8]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments, AutoModelForCausalLM, AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [9]:
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/mistral-7b-bnb-4bit',
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True)

==((====))==  Unsloth 2024.10.2: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [10]:
print(model)
print(tokenizer)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((40

## LoRA with Quantized Model

In [11]:
model_unsloth = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=['q_proj',"k_proj","v_proj","o_proj","gate_proj",'up_proj',"down_proj"],
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    max_seq_length=2048
)

Unsloth 2024.10.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [23]:
training_arg = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    output_dir = 'content/unsloth_mistral',
    warmup_steps=20,
    max_steps=100,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_8bit",
    seed = 500
)

In [26]:
trainer = SFTTrainer(
    model = model_unsloth,
    train_dataset = data,
    dataset_text_field = 'text',
    tokenizer = tokenizer,
    max_seq_length = 2048,
    args = training_arg,
    )

max_steps is given, it will override any value given in num_train_epochs


In [27]:
trainer.train()

**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=100, training_loss=2.398018798828125, metrics={'train_runtime': 1157.1994, 'train_samples_per_second': 0.691, 'train_steps_per_second': 0.086, 'total_flos': 1.5956370400739328e+16, 'train_loss': 2.398018798828125, 'epoch': 0.032})

## Test it with an example

In [28]:
test = "The movie i have watched was boring, expected action packed but, was reverse."

input = tokenizer([test], return_tensors = "pt", padding = True).to("cuda")

In [35]:
model = FastLanguageModel.for_inference(model_unsloth)

In [45]:
output = model.generate(**input, max_new_tokens=100, use_cache=True)

In [49]:
tokenizer.decode(output[0], skip_special_tokens=True)

'The movie i have watched was boring, expected action packed but, was reverse. The movie was about a guy who was a cop and was a good guy, but, he was framed by a bad guy and was sent to jail. The movie was about how he got out of jail and how he got revenge on the bad guy. The movie was not good, it was boring and i was not entertained. The movie was not good, i would not recommend this movie to anyone. The movie was not good, i would not recommend this movie to anyone. The movie'

## Save this model in hugging face

In [50]:
!pip install huggingface_hub

In [52]:
from huggingface_hub import notebook_login

notebook_login()

In [53]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

events.out.tfevents.1729274271.f5f32efa9981.21542.0:   0%|          | 0.00/6.06k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Varu96/unsloth_mistral/commit/677bfb1f2b954e317fad77b82061e63a0e1bf395', commit_message='End of training', commit_description='', oid='677bfb1f2b954e317fad77b82061e63a0e1bf395', pr_url=None, pr_revision=None, pr_num=None)